<a href="https://colab.research.google.com/github/nestorap/food-not-food/blob/main/Preprocesamiento.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
! git clone https://github.com/nestorap/food-not-food.git

Cloning into 'food-not-food'...
remote: Enumerating objects: 20, done.
remote: Counting objects: 100% (20/20), done.
remote: Compressing objects: 100% (18/18), done.
remote: Total 20 (delta 7), reused 7 (delta 2), pack-reused 0
Unpacking objects: 100% (20/20), done.


In [16]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import csv
import ast
from nltk.corpus import wordnet as wn
import nltk

In [ ]:
# Descargamos palabrasgo obtener las que son de comida

nltk.download('popular')

In [18]:
# Aquí leemos los nombres de las clases de las imágenes

with open("food-not-food/imges_list.txt", "r") as f:
  images_classes = ast.literal_eval(f.read())

In [19]:
# Recorremos los items del diccionario

for k, v in images_classes.items():
  if "banana" in v:
    print(k)



954


## Nombre de comidas de nltk

In [32]:
# Obtenemos las palabras de comida del dataset de nltk
# Con esta lista podemos filtrar las imágenes para sacar aquellas fotos que sean de comida y que no
food = wn.synset("food.n.02")
food_list = list(set([w for s in food.closure(lambda s: s.hyponyms()) for w in s.lemma_names()])) #hyponyms parece ser que significa sinónimos


In [21]:
# Check para banana
"banana" in food_list

True

## Filtramos los items para que coincidan a la hora de descargar

In [37]:
# limpiamos los string para eliminar algunos signos de puntuación

lista_food = [food_item.lower().split("_") for food_item in food_list]

In [41]:
# Unimos las palabras en una misma lista. La celda de antes obtenemos una lista de listas de palabras

food_lista = [food for food_sublist in lista_food for food in food_sublist]

In [43]:
 # checkeamos

 for k,v in images_classes.items():
   if v.lower() in food_lista:
     print(k, v)

8 hen
85 quail
86 partridge
99 goose
113 snail
331 hare
339 sorrel
390 eel
599 honeycomb
723 pinwheel
923 plate
932 pretzel
937 broccoli
938 cauliflower
946 cardoon
947 mushroom
949 strawberry
950 orange
951 lemon
952 fig
954 banana
957 pomegranate
968 cup
987 corn
988 acorn


In [52]:
#ntersection = list1_as_set. intersection(list2) 

comida_supreme = [i for i in set(food_lista).intersection([v for k,v in images_classes.items()])]